## Dashboard para a gestão municipal de São José dos Campos - SP
Um(a) gestor(a) público(a) conta com você para construir um dashboard de monitoramento que mostre a situação de seu município. Quais informações devem ser acompanhadas, e como você as apresentaria? Elabore um conjunto análises e visualizações com as principais informações para esse(a) gestor(a) de 1 (um) município a sua escolha, utilizando os dados do FarolCovid e Saúde em Ordem.

In [1]:
# Importando os módulos:
import pandas as pd

import plotly
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
import cufflinks as cf

low_memory=False

In [2]:
#Leitura dos datasets utilizados na análise
df = pd.read_csv(r'impulso-caso-full.csv', sep=',')
isolamento = pd.read_csv(r'isolamento_sjc.csv', sep=',')
farol = pd.read_csv(r'farolcovid-main.csv', sep=',')

In [3]:
#Para verificar o nome das colunas do dataset
df.columns

Index(['active_cases', 'city_id', 'city_name', 'city_notification_place_type',
       'city_notification_rate', 'confirmed_cases', 'daily_cases', 'deaths',
       'epidemiological_week', 'estimated_population_2019', 'health_region_id',
       'health_region_name', 'health_region_notification_place_type',
       'health_region_notification_rate', 'infectious_period_cases', 'is_last',
       'is_repeated', 'last_available_confirmed_per_100k_inhabitants',
       'last_available_date', 'last_available_death_rate', 'last_updated',
       'new_deaths', 'notification_rate', 'order_for_place', 'place_type',
       'state_id', 'state_name', 'state_notification_rate', 'state_num_id',
       'data_last_refreshed'],
      dtype='object')

In [4]:
#Para verificar as informações do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614986 entries, 0 to 614985
Data columns (total 30 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   active_cases                                   614986 non-null  float64
 1   city_id                                        614986 non-null  int64  
 2   city_name                                      614986 non-null  object 
 3   city_notification_place_type                   610992 non-null  object 
 4   city_notification_rate                         301820 non-null  float64
 5   confirmed_cases                                614986 non-null  float64
 6   daily_cases                                    614986 non-null  float64
 7   deaths                                         614986 non-null  float64
 8   epidemiological_week                           614986 non-null  int64  
 9   estimated_population_2019            

In [5]:
# Para formatar a coluna data('last_updated'), do tipo object para o tipo datetime
df['last_updated'] = pd.to_datetime(df.last_updated)

In [6]:
# Verificar o número de linhas e colunas
df.shape

(614986, 30)

In [7]:
#Criação de um novo dataset apenas com as informações do município desejado
df_sjc =  df[df['city_name'] == 'São José dos Campos']

In [8]:
# Verificar as 5 primeiras linhas do dataset 
df_sjc.head()

,active_cases,city_id,city_name,city_notification_place_type,city_notification_rate,confirmed_cases,daily_cases,deaths,epidemiological_week,estimated_population_2019,...,last_updated,new_deaths,notification_rate,order_for_place,place_type,state_id,state_name,state_notification_rate,state_num_id,data_last_refreshed
433330,1.0,3549904,São José dos Campos,NaN,NaN,1.0,1.0,0.0,12,721944.0,...,2020-03-18,0.0,NaN,1,city,SP,São Paulo,NaN,35,2020-08-30 12:11:03
433331,1.0,3549904,São José dos Campos,NaN,NaN,1.0,0.0,0.0,12,721944.0,...,2020-03-19,0.0,NaN,2,city,SP,São Paulo,NaN,35,2020-08-30 12:11:03
433332,1.0,3549904,São José dos Campos,NaN,NaN,1.0,0.0,0.0,12,721944.0,...,2020-03-20,0.0,NaN,3,city,SP,São Paulo,NaN,35,2020-08-30 12:11:03
433333,1.0,3549904,São José dos Campos,NaN,NaN,1.0,0.0,0.0,12,721944.0,...,2020-03-21,0.0,NaN,4,city,SP,São Paulo,NaN,35,2020-08-30 12:11:03
433334,1.0,3549904,São José dos Campos,NaN,NaN,1.0,0.0,0.0,13,721944.0,...,2020-03-22,0.0,NaN,5,city,SP,São Paulo,NaN,35,2020-08-30 12:11:03


In [9]:
isolamento.head()

,Município1,Código Município IBGE,UF1,Dia,Data,Média de Índice De Isolamento(%)
0,SÃO JOSÉ DOS CAMPOS,3549904,SP,sexta-feira,8/28/2020,42
1,SÃO JOSÉ DOS CAMPOS,3549904,SP,quinta-feira,8/27/2020,43
2,SÃO JOSÉ DOS CAMPOS,3549904,SP,quarta-feira,8/26/2020,43
3,SÃO JOSÉ DOS CAMPOS,3549904,SP,terça-feira,8/25/2020,45
4,SÃO JOSÉ DOS CAMPOS,3549904,SP,segunda-feira,8/24/2020,44


In [10]:
isolamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Município1                        169 non-null    object
 1   Código Município IBGE             169 non-null    int64 
 2   UF1                               169 non-null    object
 3   Dia                               169 non-null    object
 4   Data                              169 non-null    object
 5   Média de Índice De Isolamento(%)  169 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 8.0+ KB


In [11]:
isolamento['Data'] = pd.to_datetime(isolamento.Data)

Examinando o Índice de Isolamento Social do Município de São José dos Campos(média, desvio padrão, máxima e mínima)

In [12]:
isolamento['Média de Índice De Isolamento(%)'].describe()

count    169.000000
mean      48.869822
std        5.109012
min       31.000000
25%       46.000000
50%       48.000000
75%       52.000000
max       61.000000
Name: Média de Índice De Isolamento(%), dtype: float64

In [3]:
#Filtrando apenas as informação do município no dataset FarolCovid com índices de 
#Risco de Contágio, Subnotifcações e Capacidade Hospitalar.
farol_sjc =  farol[farol['city_name'] == 'São José dos Campos']

In [12]:
farol_sjc.head()

,active_cases,author_number_beds,author_number_icu_beds,author_number_ventilators,city_id,city_name,city_notification_place_type,confirmed_cases,dday_beds_best,dday_beds_best_months,...,rt_growth,rt_ratio_week_avg,state_id,state_name,state_notification_rate,state_num_id,subnotification_classification,subnotification_rank,subnotification_rate,data_last_refreshed
3824,3710.0,DataSUS,DataSUS,DataSUS,3549904,São José dos Campos,city,12358.0,91.0,3.0,...,estabilizando,1.00125,SP,São Paulo,0.529309,35,bom,242.0,0.253698,2020-08-29 14:07:36


In [6]:
farol_sjc.columns

Index(['active_cases', 'author_number_beds', 'author_number_icu_beds',
       'author_number_ventilators', 'city_id', 'city_name',
       'city_notification_place_type', 'confirmed_cases', 'dday_beds_best',
       'dday_beds_best_months', 'dday_beds_worst', 'dday_classification',
       'deaths', 'health_region_id', 'health_region_name', 'inloco_growth',
       'inloco_last_week_7days_avg', 'inloco_ratio_week_avgs',
       'inloco_today_7days_avg', 'last_updated_inloco',
       'last_updated_number_beds', 'last_updated_number_icu_beds',
       'last_updated_number_ventilators', 'last_updated_rt',
       'last_updated_subnotification', 'notification_rate', 'number_beds',
       'number_icu_beds', 'number_ventilators', 'overall_alert', 'population',
       'rt_10days_ago_avg', 'rt_10days_ago_high', 'rt_10days_ago_low',
       'rt_10days_ago_most_likely', 'rt_17days_ago_avg', 'rt_17days_ago_high',
       'rt_17days_ago_low', 'rt_17days_ago_most_likely', 'rt_classification',
       'rt_gro

In [13]:
# Obtém o número de mortes e casos ativos agrupando por data
temp = df_sjc.groupby('last_updated')['deaths', 'active_cases'].sum().reset_index()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [14]:
#A função melt do pandas para remodelar o dataframe com variável e valor, para ter a quantidade de casos ativos e mortos.
temp = temp.melt(id_vars="last_updated", value_vars=['deaths', 'active_cases'],
                 var_name='Case', value_name='Count')

# Visualização dos dados

In [15]:
# Habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

# Definindo o renderizador:
import plotly.io as pio
pio.renderers

Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook_connected'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

In [16]:
# Cores
mortes = '#ff2e63'
ativos = '#fe9801'

#### Gráfico 'Casos ativos e mortes ao longo do tempo'
No eixo x temos as datas e no eixo y as quantidades de cada caso (que inclusive cresce ao longo do tempo como podemos observar).

In [17]:
# Criando um gráfico de área
import plotly.express as px
fig = px.area(temp, 
              x="last_updated", 
              y="Count", 
              color='Case', 
              height=600,
              title='Casos ativos e mortes ao longo do tempo',
              color_discrete_sequence = [mortes, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

#### Gráfico 'Média de Índice De Isolamento ao longo dos Meses'
O índice de isolamento social do município foi caindo ao longo dos meses. A medida de isolamento foi alterada e os comércios começaram a funcionar a partir do dia 06 de junho. No gráfico acima é possível observar que o número de casos e de mortes aumentaram a partir da segunda quinzena do mês de junho.

In [18]:
# Criando Layout:
configuracoes_layout = go.Layout(title='Média de Índice De Isolamento ao longo dos Meses',
                                 yaxis={'title':'Índice de Isolamento %'},
                                 xaxis={'title': 'Período'})

iso = go.Scatter(x=isolamento['Data'], 
                y=isolamento['Média de Índice De Isolamento(%)'],                    
                    name = 'Média de Índice De Isolamento(%)',
                    line = dict(color = '#17BECF'))

data = [iso]

# Objeto figura:
fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico:
py.iplot(fig)

#### Gráfico 'Comparação da Taxa de Isolamento no Domingo e na Segunda'
É possível observar que o índice de isolamento de domingo para segundo chega a cair 10%.


In [19]:
iso_dom =  isolamento[isolamento['Dia'] == 'domingo']
iso_seg =  isolamento[isolamento['Dia'] == 'segunda-feira']

In [20]:
# Criando Layout:
configuracoes_layout = go.Layout(title='Comparação da Taxa de Isolamento no Domingo e na Segunda',
                                 yaxis={'title':'Taxa em %'},
                                 xaxis={'title': 'Período'})

iso_domingo = go.Scatter(x=iso_dom['Data'], 
                y=iso_dom['Média de Índice De Isolamento(%)'],                    
                        name = 'Índice De Isolamento(%) aos Domingo',
                        line = dict(color = '#17BECF'))

iso_segunda = go.Scatter(x=iso_seg['Data'], 
                        y=iso_seg['Média de Índice De Isolamento(%)'],                    
                        name = 'Índice De Isolamento(%) às Segundas',
                        line = dict(color = '#7F7F7F'))

data = [iso_domingo, iso_segunda ]

# Objeto figura:
fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico:
py.iplot(fig)

#### Gráfico 'Número de casos confirmados no Município'
São José dos Campos tem em média 1700 casos confirmados de coronavírus a cada 100 mil habitantes e o montante de casos confirmados atualmente é de 12720.

In [21]:
# Criando Layout:
configuracoes_layout = go.Layout(title='Número de casos confirmados no Município',
                                 yaxis={'title':'Número de casos'},
                                 xaxis={'title': 'Período'})


confirmed_per = go.Scatter(x=df_sjc['last_updated'], 
                   y=df_sjc['last_available_confirmed_per_100k_inhabitants'],                    
                   name = 'Confirmados por 100k habitantes')

confirmed = go.Scatter(x=df_sjc['last_updated'], 
                   y=df_sjc['confirmed_cases'],                    
                   name = 'Casos Confirmados')

data = [confirmed_per, confirmed]

# Objeto figura:
fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico:
py.iplot(fig)

In [16]:
#Filtro com apenas as colunas de interesse
filtro = ['dday_beds_best','dday_beds_worst', 'dday_classification', 'inloco_growth',
          'overall_alert','rt_growth', 'rt_ratio_week_avg', 'subnotification_classification']

In [17]:
# Visualização da Tabela de Interesse
farol_sjc[filtro].T

,3824
dday_beds_best,91
dday_beds_worst,11
dday_classification,bom
inloco_growth,estabilizando
overall_alert,medio
rt_growth,estabilizando
rt_ratio_week_avg,1.00125
subnotification_classification,bom


### Tabela Para análise de Risco 

- ##### 	Numero de dias até se esgotarem os leitos (dday_beds_best) -  No melhor cenário:
**91 dias*

- ##### 	Numero de dias até se esgotarem os leitos (dday_beds_worst) -  No melhor cenário:
**11 dias*

- ##### 	Classificação quanto ao numero de dias até se esgotarem os leitos(dday_classification): 
**Bom*

- ##### Tendencia de crescimento do indice de isolamento social(inloco_growth):
**Estabilizando*

- ##### Ritmo de Contágio (rt_ratio_week_avg):
**Insatisfatório: entre 1 e 1.2, indicando o início de um comportamento exponencial de crescimento do número de pessoas doentes, com tendência a se estabilizar (rt_growth)*

- ##### Classificação quanto a subnotificação(subnotification_classification):
**Bom*


## Conclusão:
É possível observar, após as análise dos gráficos que o número de casos confirmados e número de mortes aumentou exponencialmente após a segunda quinzena de junho, que é a data que coincide com a abertura dos comércios, diminuindo assim a Taxa de Isolamento Social do município de São José dos Campos.
Para melhor avaliação é necessário analisar os gráficos e também a tabela de Análise de Risco no enfrentamento do Covid no município.